In [2]:
%%capture
!pip install widgetsnbextension
!pip install ipywidgets
!pip install voila
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix
!pip install rudalle
!pip install scikit-learn==0.13

In [15]:
import warnings 
warnings.filterwarnings('ignore')

In [16]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import os

In [9]:
layout = widgets.Layout(width='400px', height='50px')

In [10]:
model_version = widgets.ToggleButtons(options = ['Model_1', 'Model_2'], layout = layout)

In [ ]:
display(model_version)

In [11]:
checkpoint = widgets.ToggleButtons(
            options=['12','20','40',
                     '60','80','100',
                     '120','140',
                    '160','200'], description = 'Model Version',
            layout = layout,
            display='flex',
            align_items='stretch',
            style= {'description_width': 'initial'}
        )

In [ ]:
checkpoint = widgets.ToggleButtons(
            options=['20','40',
                     '60','80'], description = 'Model Version',
            layout = layout,
            display='flex',
            align_items='stretch',
            style= {'description_width': 'initial'}
        )

In [12]:
checkpoint_1 = widgets.ToggleButtons(
            options=['20','40',
                     '60','80'], description = 'Model Version',
                     layout = layout,
            display='flex',
            align_items='stretch',
        )

In [13]:
confidence = widgets.ToggleButtons(
            options=['Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'],description='Confidence',
            style= {'description_width': 'initial'})
variability = widgets.ToggleButtons(
            options=['Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'], description = 'Variability',
            style= {'description_width': 'initial'})

In [ ]:
img_amount = widgets.IntSlider(value=2,min=1,max=9,step=1,description='Image amount',style= {'description_width': 'initial'}, layout = layout)

In [ ]:
filtered = widgets.IntSlider(value=2,min=1,max=9,step=1,description='Filtered Images',style= {'description_width': 'initial'}, layout = layout)

In [ ]:
img_amount.style.handle_color = 'lightblue'
filtered.style.handle_color = 'lightblue'

In [ ]:
display(img_amount)
display(filtered)

IntSlider(value=2, description='Image amount', max=9, min=1)

IntSlider(value=2, description='Filtered Images', max=9, min=1)

In [ ]:
prompt = widgets.Text(placeholder='Input your prompt', layout = layout)

In [ ]:
if model_version.value == 'Model_1':
    display(checkpoint_1)
elif model_version.value == 'Model 2':
    display(checkpoint)

In [ ]:
display(confidence)

ToggleButtons(description='confidence', options=('Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'), value='U…

In [ ]:
display(variability)

ToggleButtons(description='variability', options=('Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'), value='…

In [ ]:
display(prompt)

Text(value='', placeholder='Input your prompt')

In [ ]:
button_run = widgets.Button(description = 'Run model', tooltip='Send',
                style={'description_width': 'initial'}, layout = layout)

In [ ]:
button_run.style.button_color = 'lightblue'

In [ ]:
output = widgets.Output()

In [ ]:
%%capture

from rudalle import get_rudalle_model, get_vae
import torch
from model.functions import generate, get_closest_training_images_by_clip

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_rudalle_model("Malevich", pretrained=True, fp16=True, device=device)

In [ ]:
%%capture
vae = get_vae().to("cuda")

In [ ]:
import os

def on_button_clicked(event):
    with output:
        clear_output()
        if model_version.value == 'Model_1':
            checkpoint_str = 'checkpoints_1'
            v = checkpoint_1.value
        elif model_version.value == 'Model_2':
            checkpoint_str = 'checkpoints'
            v = checkpoint.value
        model_path = os.path.join(f'{checkpoint_str}/lookingglass_dalle_{v}00.pt')
        if not os.path.exists('output/'):
            os.mkdir('output/')
        filepath = f'output/{prompt.value}:{checkpoint.value}:{confidence.value}:{variability.value}'
        if not os.path.exists(filepath):
            os.mkdir(filepath)
        model.load_state_dict(torch.load(model_path))
        filenames = generate(vae, model, prompt.value, confidence = confidence.value, variability = variability.value, rurealesrgan_multiplier="x1", output_filepath=filepath, num_filtered = filtered.value, image_amount = img_amount.value)
        print(f'Images saved in {filepath}')

        for image in filenames:
            img = Image(image)
            display(img)
            

In [ ]:
button_run.on_click(on_button_clicked)

NameError: name 'button_run' is not defined

In [ ]:
display(button_run, output)

In [ ]:
vbox_result = widgets.VBox([button_run, output])

Button(description='Explore training images', style=ButtonStyle(), tooltip='Send')

Output()

In [ ]:
button_run_2 = widgets.Button(description = 'Explore training images', tooltip='Send',
                style={'description_width': 'initial'}, layout = layout)

In [ ]:
button_run_2.style.button_color = 'lightblue'

In [ ]:
output = widgets.Output()

In [ ]:
def on_button_clicked_2(event):
    from PIL import Image
    with output:
        img_filename = get_closest_training_images_by_clip(prompt.value, '../images-labelled')
        img = Image.open(f'../images-labelled/{img_filename}')
        display(img.resize((int(img.width*0.3), int(img.height*0.3))))

        

In [ ]:
button_run_2.on_click(on_button_clicked_2)

In [ ]:
display(button_run_2, output)

In [ ]:
vbox_result_2 = widgets.VBox([button_run_2, output])